In [1]:
import json 
import pandas as pd
import numpy as np 
import geopy.distance

#package for flattening json in pandas df
from pandas.io.json import json_normalize

In [2]:


use_parsed_data = True
output_parsed_data_filename = 'parsed_data.csv'

calc_last_5_games = True
calc_win_loss = True
calc_win_loss_write_all = True
calc_ratings = True
use_team_keys = False
use_team_hot_encoding = False
use_team_home_away_hot_encoding = False



In [3]:
if use_parsed_data == True:
    games = pd.read_csv(output_parsed_data_filename)
    games = games.drop(['Unnamed: 0'], axis=1)
else:
    with open('games.json') as gamesJsonRaw:
        d = json.load(gamesJsonRaw)

    games = json_normalize(d['games'])


/home/gingasu/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
with open('stadiums.json') as stadiumsJsonRaw:
    stadiumsJson = json.load(stadiumsJsonRaw)

stadiums = json_normalize(stadiumsJson['stadiums'])

In [5]:


statsColumns = pd.Series(list(games.columns.values))
statsColumns = statsColumns.loc[statsColumns.str.startswith(('statsAway', 'statsHome'), na=False)]



In [8]:


# Replace any NaN with 0 in the stats
games[statsColumns] = games[statsColumns].fillna(value=-12)



In [9]:
#games['season'] = games['season'].astype('float')
games['scoreAway'] = games['scoreAway'].astype('uint8')
games['scoreHome'] = games['scoreHome'].astype('uint8')

games['statsHome.FgAtt'] = games['statsHome.FgAtt'].astype('uint8')
games['statsHome.OffReb'] = games['statsHome.OffReb'].astype('uint8')
games['statsHome.Tov'] = games['statsHome.Tov'].astype('uint8')
games['statsHome.FtAtt'] = games['statsHome.FtAtt'].astype('uint8')
games['statsHome.Pts'] = games['statsHome.Pts'].astype('uint8')

games['statsAway.FgAtt'] = games['statsAway.FgAtt'].astype('uint8')
games['statsAway.OffReb'] = games['statsAway.OffReb'].astype('uint8')
games['statsAway.Tov'] = games['statsAway.Tov'].astype('uint8')
games['statsAway.FtAtt'] = games['statsAway.FtAtt'].astype('uint8')
games['statsAway.Pts'] = games['statsAway.Pts'].astype('uint8')

#scoreQuarters          object
#teamAwayCode           object

In [10]:
if (use_parsed_data == False):

    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(index=range(0,len(games)), columns=['gamesPlayedHome', 'gamesPlayedAway'])

    # Iterate through every team
    for team in games.teamAwayCode.unique():
        season = 0

        # Iterate through each game the team is present in.
        for index, game in games[(games['teamAwayCode'] == team) | (games['teamHomeCode'] == team)].sort_values(by='season', ascending=True).iterrows():
            if season != game.season:
                gameCount = 0
                homeGameCount = 0
                season = game.season

            gameCount += 1

            # Update game count for team - whether away or home.
            if game.teamAwayCode == team:
                data.loc[index]['gamesPlayedAway'] = gameCount
            else:
                data.loc[index]['gamesPlayedHome'] = gameCount

    data['gamesPlayedAway'] = data['gamesPlayedAway'].astype('uint8')
    data['gamesPlayedHome'] = data['gamesPlayedHome'].astype('uint8')

    # Append the results to the dataset
    games = pd.merge(games, data, left_index=True, right_index=True)

In [11]:
games[['teamHomeCode', 'teamAwayCode', 'season', 'gamesPlayedHome', 'gamesPlayedAway']].sample(5)

,teamHomeCode,teamAwayCode,season,gamesPlayedHome,gamesPlayedAway
8676,CLE,BRO,2014,77,21
7566,TOR,MIL,2013,49,43
257,MIA,BOS,2015,58,48
6413,SAC,ATL,2013,54,28
8043,CLE,ATL,2014,5,26


In [12]:


def calc_percentage(x, y):
    return 0 if y == 0 else (x / y)



In [13]:
if (use_parsed_data == False) & (calc_win_loss == True):
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(
        index=range(0,len(games)), 
        columns=[
            'totalGamesHome', 'totalWinsHome', 'homeGamesHome', 'homeWinsHome', 'awayGamesHome', 'awayWinsHome',
            'percentageTotalWinHome', 'percentageHomeWinHome', 'percentageAwayWinHome',
            'totalGamesAway', 'totalWinsAway', 'homeGamesAway', 'homeWinsAway', 'awayGamesAway', 'awayWinsAway',
            'percentageTotalWinAway', 'percentageHomeWinAway', 'percentageAwayWinAway'
        ]
    )

    # Iterate through each game.
    for index, game in games.iterrows():

        # Home Team
        # ---------
        
        historicGames = games[(games['season'] == game.season) & (games['gamesPlayedHome'] < game.gamesPlayedHome) & ((games['teamHomeCode'] == game.teamHomeCode) | (games['teamAwayCode'] == game.teamHomeCode))].sort_values(by='gamesPlayedHome', ascending=True)
       
        homeGamesHome = len(historicGames[(historicGames['teamHomeCode'] == game['teamHomeCode'])])        
        homeWinsHome = len(historicGames[(historicGames['teamHomeCode'] == game['teamHomeCode']) & (games["scoreAway"] < games["scoreHome"])])
        
        awayGamesHome = len(historicGames[(historicGames['teamAwayCode'] == game['teamHomeCode'])])        
        awayWinsHome = len(historicGames[(historicGames['teamAwayCode'] == game['teamHomeCode']) & (games["scoreAway"] > games["scoreHome"])])

        #totalGames = len(historicGames);
        #totalWins = len(
        #    historicGames[
        #        (historicGames['teamHomeCode'] == game['teamHomeCode']) & (games["scoreAway"] < games["scoreHome"]) |
        #        (historicGames['teamAwayCode'] == game['teamHomeCode']) & (games["scoreAway"] > games["scoreHome"])
        #    ]
        #)
        
        totalGamesHome = homeGamesHome + awayGamesHome
        totalWinsHome = homeWinsHome + awayWinsHome
        
        percentageTotalWinHome = round(calc_percentage(totalWinsHome, totalGamesHome), 3)
        percentageHomeWinHome = round(calc_percentage(homeWinsHome, homeGamesHome), 3)
        percentageAwayWinHome = round(calc_percentage(awayWinsHome, awayGamesHome), 3)
        
        # print('HOME', game['gamesPlayedHome'], 'TG', totalGamesHome, 'W', totalWinsHome, 'HGP', homeGamesHome, 'HW', homeWinsHome, 'AGP', awayGamesHome, 'AW', awayWinsHome)
        # print('HOME', 'W%', percentageTotalWinHome, 'HW%', percentageHomeWinHome, 'AW%', percentageAwayWinHome)
        
        # Away Team
        # ---------
        
        historicGames = games[(games['season'] == game.season) & (games['gamesPlayedAway'] < game.gamesPlayedAway) & ((games['teamAwayCode'] == game.teamAwayCode) | (games['teamHomeCode'] == game.teamAwayCode))].sort_values(by='gamesPlayedAway', ascending=True)
       
        homeGamesAway = len(historicGames[(historicGames['teamHomeCode'] == game['teamAwayCode'])])        
        homeWinsAway = len(historicGames[(historicGames['teamHomeCode'] == game['teamAwayCode']) & (games["scoreAway"] < games["scoreHome"])])
        
        awayGamesAway = len(historicGames[(historicGames['teamAwayCode'] == game['teamAwayCode'])])        
        awayWinsAway = len(historicGames[(historicGames['teamAwayCode'] == game['teamAwayCode']) & (games["scoreAway"] > games["scoreHome"])])

        #totalGames = len(historicGames);
        #totalWins = len(
        #    historicGames[
        #        (historicGames['teamHomeCode'] == game['teamHomeCode']) & (games["scoreAway"] < games["scoreHome"]) |
        #        (historicGames['teamAwayCode'] == game['teamHomeCode']) & (games["scoreAway"] > games["scoreHome"])
        #    ]
        #)
        
        totalGamesAway = homeGamesAway + awayGamesAway
        totalWinsAway = homeWinsAway + awayWinsAway
        
        percentageTotalWinAway = round(calc_percentage(totalWinsAway, totalGamesAway), 3)
        percentageHomeWinAway = round(calc_percentage(homeWinsAway, homeGamesAway), 3)
        percentageAwayWinAway = round(calc_percentage(awayWinsAway, awayGamesAway), 3)
        
        # print('AWAY', game['gamesPlayedAway'], 'TG', totalGamesAway, 'W', totalWinsAway, 'HGP', homeGamesAway, 'HW', homeWinsAway, 'AGP', awayGamesAway, 'AW', awayWinsAway)
        # print('AWAY', 'W%', percentageTotalWinAway, 'HW%', percentageHomeWinAway, 'AW%', percentageAwayWinAway)
        
        # ---------    
            
        # Update the row with the history
        data.loc[index] = [
            totalGamesHome, totalWinsHome, homeGamesHome, homeWinsHome, awayGamesHome, awayWinsHome,
            percentageTotalWinHome, percentageHomeWinHome, percentageAwayWinHome,
            totalGamesAway, totalWinsAway, homeGamesAway, homeWinsAway, awayGamesAway, awayWinsAway,
            percentageTotalWinAway, percentageHomeWinAway, percentageAwayWinAway
        ]

    # Add results to the dataset
    if (calc_win_loss_write_all == True):
        games = pd.merge(games, data, left_index=True, right_index=True)
        games[['teamHomeCode', 'teamAwayCode', 'percentageTotalWinHome', 'percentageHomeWinHome', 'percentageAwayWinHome', 'percentageTotalWinAway', 'percentageHomeWinAway', 'percentageAwayWinAway']].sample(5)
    else:
        games = pd.merge(games, data[['percentageHomeWinHome', 'percentageAwayWinAway']], left_index=True, right_index=True)
        games[['teamHomeCode', 'teamAwayCode', 'percentageHomeWinHome', 'percentageAwayWinAway']].sample(5)

In [14]:
if (use_parsed_data == False) & (calc_last_5_games == True):
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(index=range(0,len(games)), columns=['lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome'])

    # Iterate through each game where the teams have played at least 5 games.
    for index, game in games.iterrows():

        # Get the last five games for the team.
        last5games = games[(games['season'] == game.season) & (games['gamesPlayedHome'] < game.gamesPlayedHome) & ((games['teamAwayCode'] == game.teamHomeCode) | (games['teamHomeCode'] == game.teamHomeCode))].sort_values(by='gamesPlayedHome', ascending=False).head(5)

        if len(last5games) == 5:
            lastGame1WinHome = last5games["scoreAway"].iloc[0] < last5games["scoreHome"].iloc[0]
            lastGame2WinHome = last5games["scoreAway"].iloc[1] < last5games["scoreHome"].iloc[1]
            lastGame3WinHome = last5games["scoreAway"].iloc[2] < last5games["scoreHome"].iloc[2]
            lastGame4WinHome = last5games["scoreAway"].iloc[3] < last5games["scoreHome"].iloc[3]
            lastGame5WinHome = last5games["scoreAway"].iloc[4] < last5games["scoreHome"].iloc[4]
            #print(lastGame1WinHome, lastGame2WinHome, lastGame3WinHome, lastGame4WinHome, lastGame5WinHome)

            lastGame1AtHomeHome = last5games["teamHomeCode"].iloc[0] == game.teamHomeCode
            lastGame2AtHomeHome = last5games["teamHomeCode"].iloc[1] == game.teamHomeCode
            lastGame3AtHomeHome = last5games["teamHomeCode"].iloc[2] == game.teamHomeCode
            lastGame4AtHomeHome = last5games["teamHomeCode"].iloc[3] == game.teamHomeCode
            lastGame5AtHomeHome = last5games["teamHomeCode"].iloc[4] == game.teamHomeCode
            #print(lastGame1AtHomeHome, lastGame2AtHomeHome, lastGame3AtHomeHome, lastGame4AtHomeHome, lastGame5AtHomeHome)

            # Update the row with the history
            data.loc[index] = [lastGame1WinHome, lastGame1AtHomeHome, lastGame2WinHome, lastGame2AtHomeHome, lastGame3WinHome, lastGame3AtHomeHome, lastGame4WinHome, lastGame4AtHomeHome, lastGame5WinHome, lastGame5AtHomeHome]

    # Convert types 
    data['lastGame1WinHome'] = data['lastGame1WinHome'].astype('bool')
    data['lastGame2WinHome'] = data['lastGame2WinHome'].astype('bool')
    data['lastGame3WinHome'] = data['lastGame3WinHome'].astype('bool')
    data['lastGame4WinHome'] = data['lastGame4WinHome'].astype('bool')
    data['lastGame5WinHome'] = data['lastGame5WinHome'].astype('bool')
    data['lastGame1AtHomeHome'] = data['lastGame1AtHomeHome'].astype('bool')
    data['lastGame2AtHomeHome'] = data['lastGame2AtHomeHome'].astype('bool')
    data['lastGame3AtHomeHome'] = data['lastGame3AtHomeHome'].astype('bool')
    data['lastGame4AtHomeHome'] = data['lastGame4AtHomeHome'].astype('bool')
    data['lastGame5AtHomeHome'] = data['lastGame5AtHomeHome'].astype('bool')

    # Add results to the dataset
    games = pd.merge(games, data, left_index=True, right_index=True)
    
    games[['teamHomeCode', 'teamAwayCode', 'lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome']].sample(5)


In [15]:
if (use_parsed_data == False) & (calc_last_5_games == True):
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(index=range(0,len(games)), columns=['lastGame1WinAway', 'lastGame1AtHomeAway', 'lastGame2WinAway', 'lastGame2AtHomeAway', 'lastGame3WinAway', 'lastGame3AtHomeAway', 'lastGame4WinAway', 'lastGame4AtHomeAway', 'lastGame5WinAway', 'lastGame5AtHomeAway'])

    # Iterate through each game where the teams have played at least 5 games.
    for index, game in games.iterrows():

        # Get the last five games for the team.
        last5games = games[(games['season'] == game.season) & (games['gamesPlayedAway'] < game.gamesPlayedAway) & ((games['teamAwayCode'] == game.teamAwayCode) | (games['teamHomeCode'] == game.teamAwayCode))].sort_values(by='gamesPlayedAway', ascending=False).head(5)

        if len(last5games) == 5:
            lastGame1WinAway = last5games["scoreAway"].iloc[0] < last5games["scoreHome"].iloc[0]
            lastGame2WinAway = last5games["scoreAway"].iloc[1] < last5games["scoreHome"].iloc[1]
            lastGame3WinAway = last5games["scoreAway"].iloc[2] < last5games["scoreHome"].iloc[2]
            lastGame4WinAway = last5games["scoreAway"].iloc[3] < last5games["scoreHome"].iloc[3]
            lastGame5WinAway = last5games["scoreAway"].iloc[4] < last5games["scoreHome"].iloc[4]
            #print(lastGame1WinAway, lastGame2WinAway, lastGame3WinAway, lastGame4WinAway, lastGame5WinAway)

            lastGame1AtHomeAway = last5games["teamAwayCode"].iloc[0] == game.teamAwayCode
            lastGame2AtHomeAway = last5games["teamAwayCode"].iloc[1] == game.teamAwayCode
            lastGame3AtHomeAway = last5games["teamAwayCode"].iloc[2] == game.teamAwayCode
            lastGame4AtHomeAway = last5games["teamAwayCode"].iloc[3] == game.teamAwayCode
            lastGame5AtHomeAway = last5games["teamAwayCode"].iloc[4] == game.teamAwayCode
            #print(lastGame1AtHomeAway, lastGame2AtHomeAway, lastGame3AtHomeAway, lastGame4AtHomeAway, lastGame5AtHomeAway)

            # Update the row with the history
            data.loc[index] = [lastGame1WinAway, lastGame1AtHomeAway, lastGame2WinAway, lastGame2AtHomeAway, lastGame3WinAway, lastGame3AtHomeAway, lastGame4WinAway, lastGame4AtHomeAway, lastGame5WinAway, lastGame5AtHomeAway]

    # Convert types         
    data['lastGame1WinAway'] = data['lastGame1WinAway'].astype('bool')
    data['lastGame2WinAway'] = data['lastGame2WinAway'].astype('bool')
    data['lastGame3WinAway'] = data['lastGame3WinAway'].astype('bool')
    data['lastGame4WinAway'] = data['lastGame4WinAway'].astype('bool')
    data['lastGame5WinAway'] = data['lastGame5WinAway'].astype('bool')
    data['lastGame1AtHomeAway'] = data['lastGame1AtHomeAway'].astype('bool')
    data['lastGame2AtHomeAway'] = data['lastGame2AtHomeAway'].astype('bool')
    data['lastGame3AtHomeAway'] = data['lastGame3AtHomeAway'].astype('bool')
    data['lastGame4AtHomeAway'] = data['lastGame4AtHomeAway'].astype('bool')
    data['lastGame5AtHomeAway'] = data['lastGame5AtHomeAway'].astype('bool')

    # Add results to the dataset
    games = pd.merge(games, data, left_index=True, right_index=True)
    
    games[['teamHomeCode', 'teamAwayCode', 'lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome']].sample(5)

In [17]:
if (use_parsed_data == False) & (calc_ratings == True):
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(index=range(0,len(games)), columns=[
        'statsHome.seasonTotalPossessions', 'statsHome.last5TotalPossessions',
        'statsHome.seasonTotalPoints', 'statsHome.last5TotalPoints',
        'statsHome.seasonOffensiveRating', 'statsHome.last5OffensiveRating',
        'statsHome.seasonDefensiveRating', 'statsHome.last5DefensiveRating',
        'statsAway.seasonTotalPossessions', 'statsAway.last5TotalPossessions',
        'statsAway.seasonTotalPoints', 'statsAway.last5TotalPoints',
        'statsAway.seasonOffensiveRating', 'statsAway.last5OffensiveRating',
        'statsAway.seasonDefensiveRating', 'statsAway.last5DefensiveRating'
    ])

    # Iterate through each game where the teams have played at least 5 games.
    for index, game in games.iterrows():

        # Get all previous games - Home
        allgames = games[(games['season'] == game.season) & (games['gamesPlayedHome'] < game.gamesPlayedHome) & ((games['teamAwayCode'] == game.teamHomeCode) | (games['teamHomeCode'] == game.teamHomeCode))].sort_values(by='gamesPlayedHome', ascending=False)

        totalPossessionsHome = allgames['statsHome.FgAtt'].sum() - allgames['statsHome.OffReb'].sum() + allgames['statsHome.Tov'].sum() + (0.4 * allgames['statsHome.FtAtt'].sum());
        last5PossessionsHome = allgames['statsHome.FgAtt'].head(5).sum() - allgames['statsHome.OffReb'].head(5).sum() + allgames['statsHome.Tov'].head(5).sum() + (0.4 * allgames['statsHome.FtAtt'].head(5).sum());

        totalPointsScoredHome = allgames['statsHome.Pts'].sum()
        last5PointsScoredHome = allgames['statsHome.Pts'].head(5).sum()

        totalOffensiveRatingHome = round(calc_percentage(totalPointsScoredHome, totalPossessionsHome) * 100, 2)
        last5OffensiveRatingHome = round(calc_percentage(last5PointsScoredHome, last5PossessionsHome) * 100, 2)

        totalPointsAgainstHome = allgames['statsHome.PtsAgainst'].sum()
        last5PointsAgainstHome = allgames['statsHome.PtsAgainst'].head(5).sum()

        totalDefensiveRatingHome = round(calc_percentage(totalPointsAgainstHome, totalPossessionsHome) * 100, 2)
        last5DefensiveRatingHome = round(calc_percentage(last5PointsAgainstHome, last5PossessionsHome) * 100, 2)
        
        # Get all previous games - Away
        allgames = games[(games['season'] == game.season) & (games['gamesPlayedAway'] < game.gamesPlayedAway) & ((games['teamAwayCode'] == game.teamAwayCode) | (games['teamHomeCode'] == game.teamAwayCode))].sort_values(by='gamesPlayedAway', ascending=False)

        totalPossessionsAway = allgames['statsHome.FgAtt'].sum() - allgames['statsHome.OffReb'].sum() + allgames['statsHome.Tov'].sum() + (0.4 * allgames['statsHome.FtAtt'].sum());
        last5PossessionsAway = allgames['statsHome.FgAtt'].head(5).sum() - allgames['statsHome.OffReb'].head(5).sum() + allgames['statsHome.Tov'].head(5).sum() + (0.4 * allgames['statsHome.FtAtt'].head(5).sum());

        totalPointsScoredAway = allgames['statsHome.Pts'].sum()
        last5PointsScoredAway = allgames['statsHome.Pts'].head(5).sum()

        totalOffensiveRatingAway = round(calc_percentage(totalPointsScoredAway, totalPossessionsAway) * 100, 2)
        last5OffensiveRatingAway = round(calc_percentage(last5PointsScoredAway, last5PossessionsAway) * 100, 2)

        totalPointsAgainstAway = allgames['statsHome.PtsAgainst'].sum()
        last5PointsAgainstAway = allgames['statsHome.PtsAgainst'].head(5).sum()

        totalDefensiveRatingAway = round(calc_percentage(totalPointsAgainstAway, totalPossessionsAway) * 100, 2)
        last5DefensiveRatingAway = round(calc_percentage(last5PointsAgainstAway, last5PossessionsAway) * 100, 2)

        data.loc[index] = [
            totalPossessionsHome, last5PossessionsHome, 
            totalPointsScoredHome, last5PointsScoredHome, 
            totalOffensiveRatingHome, last5OffensiveRatingHome,
            totalDefensiveRatingHome, last5DefensiveRatingHome,
            totalPossessionsAway, last5PossessionsAway, 
            totalPointsScoredAway, last5PointsScoredAway, 
            totalOffensiveRatingAway, last5OffensiveRatingAway,
            totalDefensiveRatingAway, last5DefensiveRatingAway
        ]

        # Update the row with the history
        # data.loc[index] = [lastGame1WinAway, lastGame1AtHomeAway, lastGame2WinAway, lastGame2AtHomeAway, lastGame3WinAway, lastGame3AtHomeAway, lastGame4WinAway, lastGame4AtHomeAway, lastGame5WinAway, lastGame5AtHomeAway]

    data.sample(5)

    # Convert types
    # data['lastGame1WinAway'] = data['lastGame1WinAway'].astype('bool')

    # Add results to the dataset
    games = pd.merge(games, data, left_index=True, right_index=True)

    #games[['teamHomeCode', 'teamAwayCode', 'lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome']].sample(5)

In [18]:

coords_1 = (37.750267, -122.202853)
coords_2 = (34.04303865743706, -118.26711416244507)

geopy.distance.vincenty(coords_1, coords_2).miles

337.6783477035342

In [20]:
if use_parsed_data == False:
    games.to_csv(output_parsed_data_filename, encoding='utf-8')

In [21]:
if use_team_hot_encoding == True:
    homeTeam = pd.get_dummies(games["teamHomeCode"], prefix='team')
    awayTeam = pd.get_dummies(games["teamAwayCode"], prefix='team')

    homeTeam = homeTeam.replace({0:np.nan})
    awayTeam = awayTeam.replace({0:np.nan})

    teams = homeTeam.fillna(awayTeam).fillna(0).astype('bool')
    # teams[['team_NYK', 'team_CLE', 'team_ATL', 'team_BOS']].head(2)
    
    games = pd.concat([games, teams], axis=1)
    
    games[['teamHomeCode', 'teamAwayCode', 'team_NYK', 'team_CLE']].head(2)

In [22]:
if use_team_home_away_hot_encoding == True:
    homeTeam = pd.get_dummies(games["teamHomeCode"], prefix='hometeam').astype('bool')
    awayTeam = pd.get_dummies(games["teamAwayCode"], prefix='awayteam').astype('bool')

    games = pd.concat([games, homeTeam, awayTeam], axis=1)
    
    games[['teamHomeCode', 'teamAwayCode', 'hometeam_NYK', 'hometeam_CLE', 'awayteam_NYK', 'awayteam_CLE']].head(2)


In [23]:
games = games.drop(["teamAwayCode", "teamHomeCode"], axis=1)


In [24]:
games = games.drop(["location", "scoreQuarters", "time"], axis=1)

In [25]:
games = games.drop(statsColumns, axis=1)

In [26]:
games["homeWin"] = games["scoreHome"] > games["scoreAway"]

In [27]:
games = games.drop(["scoreAway", "scoreHome"], axis=1)

In [28]:
# Remove any games that have less than a 5 game history.
if calc_last_5_games == True:
    games = games[(pd.notnull(games['lastGame1WinHome'])) & (pd.notnull(games['lastGame1AtHomeAway']))]


In [29]:


# Fill any NaN stats with 0
games.fillna(-13, inplace=True)
#games['statsHome.Ejections'].fillna(0, inplace=True)
#games['statsAway.Ejections'].fillna(0, inplace=True)



In [31]:
games[games.isnull().T.any().T]

,id,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,...,lastGame1AtHomeAway,lastGame2WinAway,lastGame2AtHomeAway,lastGame3WinAway,lastGame3AtHomeAway,lastGame4WinAway,lastGame4AtHomeAway,lastGame5WinAway,lastGame5AtHomeAway,homeWin


In [32]:
train=games

In [33]:
games.shape

(9068, 45)

In [34]:
Y_train = train["homeWin"]

In [35]:
train = train.drop(["id","homeWin"], axis=1)

In [36]:
train.shape

(9068, 43)

In [37]:
import xgboost as xgb

In [38]:
from sklearn.cross_validation import  train_test_split
from sklearn.metrics import accuracy_score
for cross  in range(5):
    X_train, X_test, y_train, y_test = train_test_split(train, Y_train, test_size=0.20, random_state=cross)
    model=xgb.XGBClassifier(n_estimators=250)
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
    print(accuracy_score(y_test,pred))
    
     

/home/gingasu/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.685777287762
0.692943770673
0.656560088203
0.677508269019
0.6802646086


In [39]:
import numpy as np
import GPy
import GPyOpt
from xgboost import XGBClassifier
from sklearn.cross_validation import cross_val_score
y=Y_train.astype(int)

In [40]:
def f(parameters):
    parameters = parameters[0]
    score = -cross_val_score(
                XGBClassifier(learning_rate=parameters[0],
                              max_depth=int(parameters[2]),
                              n_estimators=int(parameters[3]),
                              gamma=int(parameters[1]),
                              min_child_weight = parameters[4]), 
                train, y, scoring='accuracy').mean()
    score = np.array(score)
    return score
baseline = -cross_val_score(XGBClassifier(), train, y, scoring='accuracy').mean()
baseline

-0.6788712816418343

In [41]:
bounds = [
            {'name': 'learning_rate', 'type': 'continuous', 'domain': (1e-5, 1)},
            {'name': 'gamma', 'type': 'continuous', 'domain': (1e-5, 5)},
            {'name': 'max_depth', 'type': 'discrete', 'domain': (1, 50)},
            {'name': 'n_estimators', 'type': 'discrete', 'domain': (1, 2000)},
            {'name': 'min_child_weight', 'type': 'discrete', 'domain': (1, 10)}
         ]

np.random.seed(777)
optimizer = GPyOpt.methods.BayesianOptimization(f=f, domain=bounds,
                                                acquisition_type ='MPI',
                                                acquisition_par = 0.1,
                                                exact_eval=True)
max_iter = 100
max_time = 100
optimizer.run_optimization(max_iter, max_time)

In [42]:
np.min(optimizer.Y)

-0.683833349424396

In [45]:
import os
 
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn.base import clone
from sklearn.pipeline import _name_estimators
from sklearn.metrics import r2_score
from __future__ import division
import pandas as pd 
import  random as random
import numpy as np
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
import heapq
from sklearn.linear_model import  LinearRegression,BayesianRidge
from sklearn.ensemble import  RandomForestRegressor,RandomForestClassifier
from sklearn.datasets import load_digits
from sklearn.ensemble import *
import math
from sklearn import metrics
from sklearn.tree import *
 
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
 
class BadErrorTreshold(Exception):
    pass
 
class EnsembleClassifier(object):
    ''''
    ensembleClassifier is a class of ensembles generated by a  fix number of  iterations.This class is abstract 
    meaning  it  is not possible  to create instances  of it.,,stochasticLearning''  has interface  role. '''
    def __init__(self,regressors,sampling,iterations,test,X,y,k_fold):
        ''''intialize ensemble with  near all  parameters- see that   some of them are  intialized  by other methods'''
        self.sampling=sampling#dropout(cv) ratio-percentage
        if (self.sampling<=0.0 or self.sampling>=1.0):
            raise ValueError('Sampling  must be percent like value between[0,1) ex 0.75, you introduced :',self.sampling)
 
 
        self.test=test#final test set
        self.X=X#original train features
        self.y=y#original train outcome
        self.iterations=iterations#number of  iterations in the  stochastic ensemble
        self.regressors=regressors#ensemble models  for  each iteration      
        self.randomIndexes=[ [] for rounds in range(self.iterations)]#filled in with random dropout  by getStochasticDataSets()
        self.error=[]#error for  each averaged iteration
 
 
        self.stochasticPredictions=[ [] for  rounds in range(self.iterations)]#predictions on random drop outs
        self.finalPredictions=[]
 
        self.x_train=[]#his   next  4 rows are intialized  by getStochasticDataSets() according to random chunks
        self.y_train=[]
        self.x_test=[]
        self.y_test=[]
 
        self.new_x_ts=[]
        self.new_x_tr=[]
        self.new_test_final=[]
        self.k_folds=k_fold
        
    def returnError(self):
        '''returns : error for  each iteration-just like cross-validation'''
        return self.error
    
    def returnRandomIndexes(self):
        '''returns: random  indexes  for  each dropout'''
        return self.randomIndexes
    
    def returnstochasticPredictions(self):
        '''returns:predictions  on random dropous'''
        return self.stochasticPredictions
    
    def getStochasticDataSets(self):
        '''
        For some  potentially  iteration creates a random  idx variable representing the index  for random  dropout.
        Splits the train in  train/test(x_train,y_train,features and outcome for cross val train-x_test,y_test for cross_val test
        .Updated this train-test  variables with corresponding  values.See that  the initial vales are empty in the __init__.
        returns: idx(index of  dropout rows).
        
        '''
        x_train, x_test, y_train, y_test = train_test_split(self.X, self.y, test_size=self.sampling)
        self.x_train=x_train
        self.x_test=x_test
        self.y_train=y_train
        self.y_test=y_test
        #train_i, test_i = train_test_split(np.arange(self.X.shape[0]), train_size = self.sampling)
        #self.x_train=self.X[train_i]
        #self.y_train=self.y[train_i]
        #self.x_test=self.X[test_i]
        #self.y_test=self.y[test_i]
        #idx=test_i
        #return  idx
    
    def training(self,i):
        ''''
        For a  particular iteration ,,i''  calls getStochasticDataSets() method.In this was the randomIndexes feature is
        fiiled with corresponding index dropouts at position ,,i''.Since train/test new data sets according to cross validation
        are   filled  by getStochasticDataSets()  independet  of iteration number , this data sets will update for each new iter.
        performmed in stochasticLearning().'''
        self.getStochasticDataSets()
        #blending- we have x_train,y_train,x_test,y_test   our stochastic  dropouts for diversity
        #we are  going first  to make blend train,blend test from this dropoutsthen will see evaluation on the dropout
        clfs=self.regressors
        n_folds=self.k_folds
        skf=list(StratifiedKFold(self.y_train,n_folds))
        blend_tr=np.zeros((self.x_train.shape[0],len(clfs)))
        blend_ts=np.zeros((self.x_test.shape[0],len(clfs)))#for loca al evaluation
        blend_final_test=np.zeros((self.test.shape[0],len(clfs)))
 
        for  j , clf in  enumerate(clfs):
            print(clf)
            blend_ts_j=np.zeros((self.x_test.shape[0],len(skf)))
            blend_final_j=np.zeros((self.test.shape[0],len(skf)))
            for i ,(tr,ts) in enumerate(skf):
                print("fold",i)
                x_tr=self.x_train[tr]
                y_tr=self.y_train[tr]
                x_ts=self.x_train[ts]
                y_ts=self.y_train[ts]
                clf.fit(x_tr,y_tr)
                y_sub=clf.predict_proba(x_ts)[:,1]
                blend_tr[ts,j]=y_sub
                blend_ts_j[:,i]=clf.predict_proba(self.x_test)[:,1]
                blend_final_j[:,i]=clf.predict_proba(self.test)[:,1]
            blend_ts[:,j]=blend_ts_j.mean(1)
            blend_final_test[:,j]=blend_final_j.mean(1)
        #blend on evaluation
        clf = LogisticRegression()
        clf.fit(blend_tr, self.y_train)
        
        pred_eval=  clf.predict_proba(blend_ts)[:,1]
        self.error.append(log_loss(self.y_test,pred_eval))
        #blend  on final test set:
        final_sub=clf.predict_proba(blend_final_test)[:,1]
        self.finalPredictions.append(final_sub)
        
        
 
        return self
    
    def getFinalPrediction(self,errorTreshold):
       raise NotImplementedError###averages models  according to best top prediction  trehsold  with different implementantions
    
    
        
    
    def stochasticLearning(self):
         raise NotImplementedError#Since EnsemebleRegressors class is abstract this will act  as an interface
                                  #and will be implemented   by  this  EnsembleClassifier sublclasses  for averaging and
                                  #weighted average
    
        
class AveragingModels(EnsembleClassifier):
    '''Inherits everything from EnsembleRegressors  class and implements stochastiLearning method  averaging models 
    for each iteration'''
 
    def stochasticLearning(self):
        
        
        for iter in range(self.iterations):
            print('Stochastic Iteration number ',iter)
            self.training(iter)
 
    def getFinalPrediction(self,errorTreshold):
        if (errorTreshold<=self.iterations and errorTreshold>0):
            
            topErrorIndex=heapq.nsmallest(errorTreshold, range(len(self.error)), self.error.__getitem__)
            finalAvg=0
            for topError in topErrorIndex:
                finalAvg=finalAvg+self.finalPredictions[topError]
            finalAvg=(finalAvg)/float(len(topErrorIndex))
            return finalAvg
        else:
            raise BadErrorTreshold
       
       